In [119]:
%run "libraries.ipynb"

common libraries loaded


In [6]:
def from_file(name):
  diff_txt = ""

  with codecs.open(name, "r", encoding="utf-8-sig") as f:
    data = json.load(f)

  return data

def list_revisions(page):
  return os.listdir("data/%s" % (page))

def load_revisions(s):
  revisions = defaultdict(dict)
  
  p = wekeypedia.WikipediaPage(s)
  
  revisions_list = list_revisions(s)
  revisions_list = map(lambda revid: revid.split(".")[0], revisions_list)
  
  revisions = { revid : from_file("data/%s/%s.json" % (s, revid)) for revid in revisions_list }
  
  return revisions

revisions = load_revisions("Love")

len(revisions)

6324

In [137]:
def detect_love(revid):
  result = {
    "revid": revid,
    "love": [],
    "plusminus": {}
  }
  
  diff = revisions[revid]["diff"]["*"]
  diff = page.extract_plusminus(diff)
  
  result["plusminus"] = diff
  
  rev_index = revisions.keys()
  print "\rrevision: %s/%s" % ( rev_index.index(revid), len(rev_index)),
  
  for sentence in diff["added"]:
    pos_tags = nltk.pos_tag(nltk.word_tokenize(sentence))

    if len([ t for t in pos_tags if t[1] == "NNP" \
            and not("love" in t[0].lower())]) >= 2 \
            and 2 < len(pos_tags) < 20 \
            and "love" in sentence.lower():
      
      result["love"].append(sentence)
      print " ♥︎",

  return result

page = wekeypedia.WikipediaPage("Love")

# revlist = random.sample(revisions.keys(), 100)
revlist = revisions.keys()

results = [ detect_love(revid) for revid in revlist ]
print "\r ",

love = [ s for s in results if len(s["love"]) > 0 ]

print "♥︎" * len(love)
print len(love)

revision: 786/6324

KeyboardInterrupt: 

In [ ]:
final_result = []

for l in love:
  for s in l["love"]:
    if not("[" in s) and not("*" in s) and not("==" in s):
      final_result.append( [ l["revid"], s ] )

final_result = pd.DataFrame(final_result)
final_result.columns = ["revision id", "sentence"]
final_result.head()

print len(final_result)

final_result.to_csv("data/find_love.csv", encoding="utf-8")

At this point there is still ~70 false positives and no counting of false negatives

In [ ]:
final_result = final_result.from_csv("data/find_love-checked.csv", encoding="utf-8")
final_result = final_result.drop(final_result[final_result["false positive"] == 1].index)

print len(final_result)

In [138]:
def get_line_diff(revid):
  line = 0
  
  content = revisions[u""+str(revid)]["diff"]["*"]
  html = BeautifulSoup(content, 'html.parser')
  
  # <td colspan="2" class="diff-lineno">
  line = html.find("td", class_="diff-lineno").get_text()
  line = line.split(" ")[1]
  line = line[0:-1]
  
  return line

In [177]:
# content = page.get_revision()
content = BeautifulSoup(page.get_revision(), 'html.parser')

for l in final_result.index:
  line = get_line_diff(l)
  tag = u"<span class=\"love\" data-revision_id=\"%s\">♥︎</span>" % (l)
  
  content.insert(int(line), BeautifulSoup(tag, 'html.parser'))

display(HTML("<h1>Preview</h1>"))
display(HTML(content.prettify()))

with codecs.open("outputs/findlove.html", "w", "utf-8-sig") as f:
  f.write(content.prettify())
  f.close()